In [ ]:
# To Do ... organizar una verificación de versiones de librerias.

In [1]:
#@title #1. Library { display-mode: "form" }
import os
import re
import math
import warnings
import numpy as np
import pandas as pd
import gspread as gs
import seaborn as sns
from google.colab import files, drive, auth
from google.auth import default
from collections import Counter
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [2]:
#@title Autenticador en drive { display-mode: "form" }
#@markdown ---
# authorization in drive
drive.mount('/content/drive')
dir_path = '/content/drive/MyDrive/ETB/Codes/2022/Functions/Normalization/'
os.chdir(dir_path)
from func_callout import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title función de normalización
# library
import re
import math
import warnings
import numpy as np
import pandas as pd
from collections import Counter

# Consistencia
def aux_f(x):
    """
    Input:
    x: Int Dato de la sabada ubicadas en los teléfonos. 
    Output:
    x: Int Dato verificado con las condiciones solicitadas.
    Example:
    ...
    """
    # else:
      ## Verificar Número de teléfono
    if len(x)==7:
        return x
    if len(x)==8:
        return x[1:]
    if len(x)==9 and x[0:2]=='57':
      return x[2:]
    if len(x)==9 and x[0:1]=='3':
        return x[2:]
    if len(x)==10 and (x[0:2]=='57' or x[0:2]=='60'):
        return x[2:]
    ## Verificar Número de celular
    if len(x)==10 and x[0]=='3':
        return x
    if len(x)==11 and x[0:2]=='13':
        return x[1:]
    elif len(x)==11 and x[1:2]=='3':
        return x[1:]
    if len(x)==12 and x[0:2]=='57':
        return x[2:]
    if len(x)==14 and (x[0:2]=='5760' or x[0:2]=='6057' or x[0:2]=='9957') and x[4:][0]=='3':
        return x[4:]
    else:
        # return np.nan
        # identificadores de reporte.
        nu_exc = ['13777777','12345678','123456','234567','12121212','11111','22222',
                '33333','44444','55555','66666','77777','88888','99999','00000']
        aux = list(Counter(x).values()) # valor auxiliar de conteo    
        max = sorted(aux, reverse=True)[0]
        # aplicar comparaciones según los requerimientos de los teléfonos.
        if len([True for i in nu_exc if i in x])>0:
            return np.nan

def TF(x):
    """
    Input:
    x: Dato de la sabada ubicadas en los teléfonos. 
    Output:
    x: Dato verificado con las condiciones solicitadas.
    Example:
    ...
    """
    aux = [float(s) for s in re.findall(r'-?\d+\.?\d*', ''.join(str(x).split()))]
    aux = [str(abs(int(float(i)))) for i in aux]
    len_aux = np.array([len(i) for i in aux])
    if aux != []:
        if len(aux[0])>10 and aux[0][0]=='3':
            chunks = len(aux[0])
            aux_ = [aux[0][i:i+10] for i in range(0, chunks, 10)]
            aux = aux[1:]+aux_
            len_aux = np.array([len(i) for i in aux])
        elif len(aux[0])>10 and aux[0][0]=='1':
            chunks = len(aux[0])
            aux_ = [aux[0][i:i+10] for i in range(0, chunks, 8)]
            aux = aux[1:]+aux_
            len_aux = np.array([len(i) for i in aux])
        if len(aux) == 1:
            return aux_f(aux[0])
        else:
            if len(np.where(len_aux==10)[0])!=0:
                return aux_f(aux[np.where(len_aux==10)[0][0]])
            elif len(np.where(len_aux==8)[0])!=0:
                return aux_f(aux[np.where(len_aux==8)[0][0]])
            else:
                return aux_f(aux[0])
    else:
        return np.nan

def NC(x):
    """
    Input:
    x: Dato de la sabada ubicadas en los teléfonos. 
    Output:
    x: Dato verificado con las condiciones solicitadas, para selección del número.
    Example:
    ...
    """
    aux = [float(s) for s in re.findall(r'-?\d+\.?\d*', ''.join(str(x['NUMERO_CONEXION']).split()))]
    aux = [str(abs(int(float(i)))) for i in aux]
    if aux != []:
        if not(math.isnan(float(aux[0]))):
            if len(aux[0])==8 and aux[0][0]=='1':
                return aux[0]
            elif len(aux[0])==10 and (aux[0][0:2]=='60' or aux[0][0:2]=='57'):
                return aux[0][2:]
            else:
                return np.nan
        else:
            return np.nan
    else:
        return np.nan

# Completitud
def completeness(x):
    """
    Input:
    x: DataFrame Sabana con las series analizar telefono_1.
    Output:
    x: DataFrame Retorna la sabana verificando completitud en número de teléfono.
    """
    return x[(x['telefono_1'].isnull())].index

# Función principal para realizar la verificación de la información almacenada en la sabana.
def main(X):
    """
    Input:
    X: DataFrame Sabana con la información completa. 
    Output:
    X: DataFrame Sabana con la información verificada aplicando Consistencia, Duplicidad y Completitud.
    """
    print('cantidad de registros: {} \n' .format(len(X)))
    
    ## Sección de Consistencia
    print('Inicio del pre-proceso... etapa 1-> Consistencia\n')
    ## Teléfono 1
    print('Inicio del pre-proceso de la columna telefono_1')
    X['telefono_1'] = X['telefono_1'].apply(lambda x: x if (isinstance(x,(float,int)) and not(math.isnan(x))) else x)  
    X['telefono_1'] = X['telefono_1'].apply(aux_f)
    X['largo_telefono_1'] = X['telefono_1'].apply(lambda x: len(x) if not(pd.isnull(x)) else x)
    print('Finalizacion del pre-proceso de la columna telefono_1')
    print('cantidad de registros {} \n' .format(len(X[X['telefono_1'].notna()])))

    ## Sección de Completitud
    print('Inicio del preproceso... etapa 2-> Completitud \n')
    index = completeness(X)
    print('cantidad de registros incompletos: {} \n' .format(len(index))) 
    print('cantidad de registros: {} \n' .format(len(X)))
    
    ## Eliminar información temporal
    X.drop(['largo_telefono_1'],axis=1, inplace=True)
    
    ## Renombrar los parámetros que se usaron para las funciones de verificación
    X.rename(columns={'telefono_1':'Telefono'},inplace=True)
    
    # Retorna la sabana depurada y organizada como se necesita en los requerimientos
    return X

## 2. Buscar el archivo y cargarlo

In [4]:
#@title ###Ubicación de archivos en el mes { vertical-output: true }
Mes = 'Febrero' #@param ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
Year= '2023' #@param ['2022', '2023']
dir_path = '/content/drive/MyDrive/ETB/Data/Actualización Normalización Data ETB/'
os.chdir(dir_path)
path = os.path.join('Data/callout/'+Year+'/'+Mes,'')

In [5]:
#@title ### Buscar archivos disponibles.
os.listdir(path)

['Base_callout_Emergia_20230227_normalizada.xlsx',
 'Base_callout_Emergia_20230227.xlsx',
 'Base_callout_Emergia_20230302 Chat experiencia.xlsx']

In [6]:
#@title ###Carga de datos { run: "auto", vertical-output: true }
Nombre_archivo = 'Base_callout_Emergia_20230302 Chat experiencia' #@param {type:"string"}
nombre_completo= [i for i in os.listdir(path) if Nombre_archivo in i][0]
df = pd.read_excel(path+nombre_completo)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NUMERO_DOCUMENTO  16000 non-null  int64 
 1   MarcaPlantaFija   16000 non-null  object
 2   FlagPlantaFija    16000 non-null  int64 
 3   MarcaPlantaMovil  16000 non-null  object
 4   FlagPlantaMovil   16000 non-null  int64 
 5   MarcaHabeasData   16000 non-null  object
 6   HABEAS_DATA       16000 non-null  object
 7   TELEFONO_MOVIL_H  16000 non-null  object
 8   NOMBRE_CLIENTE    16000 non-null  object
dtypes: int64(3), object(6)
memory usage: 1.1+ MB


In [ ]:
df.NUMERO_DOCUMENTO

0          71673257
1           8434633
2        1032508182
3          52229805
4        1095949799
            ...    
15995      52313841
15996      51693521
15997    1001173805
15998      79798605
15999    1020788975
Name: NUMERO_DOCUMENTO, Length: 16000, dtype: int64

In [7]:
winback = '/content/drive/MyDrive/ETB/Data/Actualización Normalización Data ETB/Data/winback/2023/Febrero/Base_Win_Back_Emergia_Marzo2023.xlsx'
callout = '/content/drive/MyDrive/ETB/Data/Actualización Normalización Data ETB/Data/callout/2023/Febrero/Base_callout_Emergia_20230227.xlsx'
ETB_Total = '/content/drive/MyDrive/ETB/Data/Actualización bases mensuales ETB Total/Data/ETB_Total/2023/Abril/seguimiento_Mar_v4.csv'
Data_winback = pd.read_excel(winback)
Data_ETB_Total=pd.read_csv(ETB_Total)
Data_callout = pd.read_excel(callout)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,5,47,53,54,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.columns

Index(['NUMERO_DOCUMENTO', 'MarcaPlantaFija', 'FlagPlantaFija',
       'MarcaPlantaMovil', 'FlagPlantaMovil', 'MarcaHabeasData', 'HABEAS_DATA',
       'TELEFONO_MOVIL_H', 'NOMBRE_CLIENTE'],
      dtype='object')

# Verificación con Nro de documento

In [8]:
df.TELEFONO_MOVIL_H = df.TELEFONO_MOVIL_H.astype(str)
df['Tam_num'] = 0
df['Tam_num'] = df.TELEFONO_MOVIL_H.apply(lambda x: len(x))
df = df[df['Tam_num']==10]

In [9]:
df.TELEFONO_MOVIL_H  

0        3146414515
1        3003702917
2        3202627422
3        3113969612
4        3232520081
            ...    
15995    3168319009
15996    3143285828
15997    3058701734
15998    3134887889
15999    3215066350
Name: TELEFONO_MOVIL_H, Length: 15999, dtype: object

In [ ]:
#@title ###Verificador de parámetros de busqueda interna
dfo=check(df)
print('Verificación terminada')
dfo.info()

Verificación terminada
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 77 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Periodo                        75000 non-null  int64         
 1   id_base                        75000 non-null  object        
 2   nro_registro                   75000 non-null  object        
 3   campana                        75000 non-null  object        
 4   proveedor                      75000 non-null  object        
 5   NOMBRE_CLIENTE                 74926 non-null  object        
 6   TIPO_DOCUMENTO                 75000 non-null  object        
 7   NUMERO_DE_DOCUMENTO            75000 non-null  object        
 8   direccion_correo_electronico   74491 non-null  object        
 9   SegUEN_Hog                     75000 non-null  object        
 10  cantidad_lineas                5000 non-null   float64     

In [ ]:
#@title ####Dimensión del registro
dfo.shape

(75000, 77)

## 3. Ejecutar la función de normalizacion

In [ ]:
#@title ### Normalización
X_pre = main(dfo)

cantidad de registros: 75000 

Inicio del pre-proceso... etapa 1-> Consistencia

Inicio del pre-proceso de la columna NUMERO_CONEXION
Finalizacion del pre-proceso de la columna NUMERO_CONEXION
cantidad de registros: 74994 

Inicio del pre-proceso de la columna telefono_1
Finalizacion del pre-proceso de la columna telefono_1
cantidad de registros 58974 

Inicio del pre-proceso de la columna telefono_2
Finalizacion del pre-proceso de la columna telefono_2
cantidad de registros 72874 

Inicio del pre-proceso de la columna telefono_3
Finalizacion del pre-proceso de la columna telefono_3
cantidad de registros 46380 

Inicio del pre-proceso... etapa 2-> Duplicidad 

Inicio del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 58974 

cantidad de registros telefono 2: 72874 

cantidad de registros telefono 3: 46380 

Eliminación de Duplicidad en los telefonos
Finalización del pre-proceso de Duplicidad en los telefonos
cantidad de registros telefono 1: 58974 

cantid

In [ ]:
#@title ####Cabecera del registro
X_pre.head(2)

,Periodo,id_base,nro_registro,campana,proveedor,NOMBRE_CLIENTE,TIPO_DOCUMENTO,NUMERO_DE_DOCUMENTO,direccion_correo_electronico,SegUEN_Hog,...,Negocio_seguro,Office_365,Oferta_UpGrade,Valor_UpGrade,Win_Online,hbo_max,W_OTT,fecha_habeas_data,fuente_habeas_data,oferta_adultos
0,202303,MK20230224-06-1,2023-02-276-04759,Up grade de velocidad Hogares 500,Emergia,AURA MARINA RAMIREZ CELY,CC,20249347,KR 59D 131A 60/AURA61@OUTLOOK.COM,HOGARES,...,NaN,NaN,NaN,NaN,SI,SI,,2019-07-01 19:00:00,MDM_CLIENTE,NaN
1,202303,MK20230224-06-1,2023-02-276-03310,Up grade de velocidad Hogares 500,Emergia,HECTOR HERNANDEZ ALBAÑIL,CC,17090912,KR 70D 98A 05 IN 11 AP 303/DOCUMENTOSDIGITALES...,HOGARES,...,NaN,NaN,NaN,NaN,SI,SI,,2019-07-01 19:00:00,MDM_CONTACTO,NaN


In [ ]:
#@title ####Dimensión del registro
X_pre.shape

(75000, 77)

In [ ]:
#@title Guardar sabana { vertical-output: true, form-width: "600px" }
Nombre_adicional = "_normalizada" #@param {type:"string"}
Formato = '.xlsx' #@param ['.xlsx', '.csv']
if Formato == '.xlsx':
  X_pre.to_excel(path+nombre_completo[:-5]+Nombre_adicional+'.xlsx')
else:
  X_pre.to_csv(path+nombre_completo[:-5]+Nombre_adicional+'.csv')

print('Sabana almacenada en {}, con el nombre de {}{}'.format(path,nombre_completo[:-5]+Nombre_adicional,Formato))

Sabana almacenada en Data/callout/2023/Febrero/, con el nombre de Base_callout_Emergia_20230227_normalizada.xlsx


# temporal

In [9]:
Lista_callout = ['Base_callout_Emergia_20220729.xlsx', 'Base_callout_Emergia_20220829.xlsx',
                 'Base_callout_Emergia_20220928.xlsx', 'Base_callout_Emergia_20221028.xlsx',
                 'Base_callout_Emergia_20221028 wb, avanzados.xlsx', 
                 'Base_callout_Emergia_20221028 directv2, avanzados.xlsx',
                 'Base_callout_Emergia_20221123_ap.xlsx', 'Base_callout_Emergia_20221129.xlsx',
                 'Base_callout_Emergia_20221228.xlsx']

Lista_callout1= ['Base_callout_Emergia_20230111_migraciones.xlsx', 'Base_callout_Emergia_20230112_migraciones.xlsx',
                 'Base_callout_Emergia_20230131.xlsx', 'Base_callout_Emergia_20230227.xlsx']

winback = '/content/drive/MyDrive/ETB/Data/Actualización Normalización Data ETB/Data/callout/'

In [20]:
dft_ct.columns

Index(['Periodo', 'id_base', 'nro_registro', 'campana', 'proveedor',
       'NOMBRE_CLIENTE', 'TIPO_DOCUMENTO', 'NUMERO_DE_DOCUMENTO',
       'direccion_correo_electronico', 'SegUEN_Hog', 'cantidad_lineas',
       'NUMERO_CONEXION', 'telefono_1', 'telefono_2', 'telefono_3',
       'departamento', 'ciudad_de_instalacion', 'ESTRATO', 'Plan_LT_Corto',
       'promedio_fact', 'Plan_sugerido_1', 'vlr_oferta1', 'Plan_sugerido_2',
       'vlr_oferta2', 'Plan_sugerido_3', 'vlr_oferta3', 'DIR_GEN', 'NODO',
       'COBERTURA_LOTE', 'NOMBRE_PLAN_LD', 'NOMBRE_PLAN_LT', 'NOMBRE_PLAN_BA',
       'NOMBRE_PLANTV', 'TARIFA_LT_CON_IVA', 'TARIFA_BA_CON_IVA',
       'TARIFA_TV_CON_IVA', 'TARIFA_TOTAL_CON_IVA',
       'PERMANENCIA_ANIOS_LT_SERVICIO', 'PRODUCTO_MOVIL', 'EXCLUSION',
       'Mezcla_', 'VELOCIDAD_KB', 'Celular_ETB', 'vlr_oferta_ingresos',
       'Cuenta_Padre_Final', 'Riesgo_Comport', 'RESULTADO_FB', 'USER_ID',
       'INICIO_SERVICIO_TV', 'SEGUNDOS_CANALES', 'ACTIVIDAD_ECONOMICA',
       'DES

In [27]:
dft_ct.CELULAR_ADIC_1[dft_ct.CELULAR_ADIC_1.notna()] = dft_ct.CELULAR_ADIC_1[dft_ct.CELULAR_ADIC_1.notna()].replace(' ', 'nan')

<ipython-input-27-17f1839966fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft_ct.CELULAR_ADIC_1[dft_ct.CELULAR_ADIC_1.notna()] = dft_ct.CELULAR_ADIC_1[dft_ct.CELULAR_ADIC_1.notna()].replace(' ', 'nan')


In [30]:
#@title Comparación de la base de datos si se encuentra duplicidad entre df y callout

year1 = ['2022']
mes1  = ['Julio', 'Agosto', 'Septiembre', 'Octubre', 'Octubre', 'Octubre', 'Octubre', 'Noviembre', 'Diciembre', 'Diciembre', 'Diciembre']
year2 = ['2023']
mes2  = ['Enero', 'Febrero']

i = 0
for y in year1:
  for m in mes1:
    Data_winback = pd.read_excel(winback+'/'+y+'/'+m+'/'+Lista_callout[i])
    # Datos
    # dft_st = df_t[df_t_.TELEFONO.isna()]
    Data_winback.CELULAR_ADIC_1[Data_winback.CELULAR_ADIC_1.notna()] = Data_winback.CELULAR_ADIC_1[Data_winback.CELULAR_ADIC_1.notna()].replace(' ', 'nan')
    dft_ct = Data_winback[Data_winback.CELULAR_ADIC_1.notna()]#[df.TELEFONO.notna()]
    dft_ct.reset_index(drop = True, inplace= True)
    # dfm_st = df_m[df_m.TELEFONO.isna()]
    dfm_ct = df#[Data_winback.TELEFONO.notna()]
    dfm_ct.reset_index(drop = True, inplace= True)
    x1 = dft_ct['NUMERO_DE_DOCUMENTO'].tolist()
    y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
    # x2 = dft_ct['TELEFONO'].tolist()
    # y2 = dfm_ct['TELEFONO'].tolist()
    ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
    # ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
    # indx = list(set(ind_doc) & set(ind_con))
    print(len(ind_doc))
    # nuevos
    df_New = dfm_ct.drop(ind_doc, axis=0)
    print('Registros nuevos: {}'.format(df_New.shape))
    i+=1

i = 0
for y in year2:
  for m in mes2:
    Data_winback = pd.read_excel(winback+'/'+y+'/'+m+'/'+Lista_callout1[i])
    Data_winback.CELULAR_ADIC_1[Data_winback.CELULAR_ADIC_1.notna()] = Data_winback.CELULAR_ADIC_1[Data_winback.CELULAR_ADIC_1.notna()].replace(' ', 'nan')
    # Datos
    # dft_st = df_t[df_t_.TELEFONO.isna()]
    dft_ct = Data_winback[Data_winback.CELULAR_ADIC_1.notna()]#[df.TELEFONO.notna()
    dft_ct.reset_index(drop = True, inplace= True)
    # dfm_st = df_m[df_m.TELEFONO.isna()]
    dfm_ct = df#[Data_winback.TELEFONO.notna()]
    dfm_ct.reset_index(drop = True, inplace= True)
    x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
    y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
    # x2 = dft_ct['TELEFONO'].tolist()
    # y2 = dfm_ct['TELEFONO'].tolist()
    ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
    # ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
    # indx = list(set(ind_doc) & set(ind_con))
    print(len(ind_doc))
    # nuevos
    df_New = dfm_ct.drop(ind_doc, axis=0)
    print('Registros nuevos: {}'.format(df_New.shape))
    i+=1

<ipython-input-30-778ee5880bc3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_winback.CELULAR_ADIC_1[Data_winback.CELULAR_ADIC_1.notna()] = Data_winback.CELULAR_ADIC_1[Data_winback.CELULAR_ADIC_1.notna()].replace(' ', 'nan')


0
Registros nuevos: (15999, 10)
0
Registros nuevos: (15999, 10)
0
Registros nuevos: (15999, 10)
0
Registros nuevos: (15999, 10)
0
Registros nuevos: (15999, 10)
5
Registros nuevos: (15994, 10)
12
Registros nuevos: (15987, 10)
0
Registros nuevos: (15999, 10)
0
Registros nuevos: (15999, 10)


IndexError: ignored

In [33]:
i

9

# winback

In [ ]:
Lista_winback=['Base_Win_Back_Emergia_Agosto2022.xlsx','Base_Win_Back_Emergia_Septiembre2022.xlsx',
               'Base_Win_Back_Emergia_Noviembre2022.xlsx','Base_Win_Back_Emergia_Diciembre2022.xlsx',
               'BBDD Winback v2 Enero 2023.xlsx',
               ]
Lista_winback1=['Base_Win_Back_Emergia_Febrero2023.xlsx','Base_Win_Back_Emergia_Marzo2023.xlsx']
              
winback = '/content/drive/MyDrive/ETB/Data/Actualización Normalización Data ETB/Data/winback/'

In [ ]:
#@title Comparación de la base de datos si se encuentra duplicidad entre df y winback

year1 = ['2022']
mes1  = ['Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
year2 = ['2023']
mes2  = ['Enero', 'Febrero']

i = 0
for y in year1:
  for m in mes1:
    Data_winback = pd.read_excel(winback+'/'+y+'/'+m+'/'+Lista_winback[i])
    # Datos
    # dft_st = df_t[df_t_.TELEFONO.isna()]
    dft_ct = Data_winback#[df.TELEFONO.notna()]
    dft_ct.reset_index(drop = True, inplace= True)
    # dfm_st = df_m[df_m.TELEFONO.isna()]
    dfm_ct = df#[Data_winback.TELEFONO.notna()]
    dfm_ct.reset_index(drop = True, inplace= True)
    x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
    y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
    # x2 = dft_ct['TELEFONO'].tolist()
    # y2 = dfm_ct['TELEFONO'].tolist()
    ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
    # ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
    # indx = list(set(ind_doc) & set(ind_con))
    print(len(ind_doc))
    # nuevos
    df_New = dfm_ct.drop(ind_doc, axis=0)
    print('Registros nuevos: {}'.format(df_New.shape))
    i+=1

i = 0
for y in year2:
  for m in mes2:
    Data_winback = pd.read_excel(winback+'/'+y+'/'+m+'/'+Lista_winback1[i])
    # Datos
    # dft_st = df_t[df_t_.TELEFONO.isna()]
    dft_ct = Data_winback#[df.TELEFONO.notna()]
    dft_ct.reset_index(drop = True, inplace= True)
    # dfm_st = df_m[df_m.TELEFONO.isna()]
    dfm_ct = df#[Data_winback.TELEFONO.notna()]
    dfm_ct.reset_index(drop = True, inplace= True)
    x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
    y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
    # x2 = dft_ct['TELEFONO'].tolist()
    # y2 = dfm_ct['TELEFONO'].tolist()
    ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
    # ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
    # indx = list(set(ind_doc) & set(ind_con))
    print(len(ind_doc))
    # nuevos
    df_New = dfm_ct.drop(ind_doc, axis=0)
    print('Registros nuevos: {}'.format(df_New.shape))
    i+=1

In [ ]:
Data_winback[Data_winback.TelefonoMovilCtaCliente.notna()]

In [ ]:
#@title Comparación de la base de datos si se encuentra duplicidad entre df y winback
year1 = ['2022']
mes1  = ['Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

i = 0
for y in year1:
  for m in mes1:
    Data_winback = pd.read_excel(winback+'/'+y+'/'+m+'/'+Lista_winback[i])
    # Datos
    # dft_st = df_t[df_t_.TELEFONO.isna()]
    dft_ct = Data_winback[Data_winback.TelefonoMovilCtaCliente.notna()]
    dft_ct.reset_index(drop = True, inplace= True)
    # dfm_st = df_m[df_m.TELEFONO.isna()]
    dfm_ct = df#[Data_winback.TELEFONO.notna()]
    dfm_ct.reset_index(drop = True, inplace= True)
    x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
    y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
    x2 = dft_ct['TelefonoMovilCtaCliente'].tolist()
    y2 = dfm_ct['TELEFONO_MOVIL_H'].tolist()
    ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
    ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
    indx = list(set(ind_doc) & set(ind_con))
    # print(len(ind_doc))
    # nuevos
    # df_New = dfm_ct.drop(ind_doc, axis=0)
    df_New = dfm_ct.drop(indx, axis=0)
    df['toque_'+y+'_'+m] = 0
    df['toque_'+y+'_'+m].iloc[indx] = 1
    print('Registros nuevos: {}'.format(df_New.shape))
    i+=1

Registros nuevos: (15999, 12)
Registros nuevos: (15999, 13)
Registros nuevos: (15947, 14)
Registros nuevos: (15999, 15)
Registros nuevos: (15986, 16)


In [ ]:
year2 = ['2023']
mes2  = ['Enero', 'Febrero']
i = 0
for y in year2:
  for m in mes2:
    Data_winback = pd.read_excel(winback+'/'+y+'/'+m+'/'+Lista_winback1[i])
    # Datos
    # dft_st = df_t[df_t_.TELEFONO.isna()]
    dft_ct = Data_winback[Data_winback.TelefonoMovilCtaCliente.notna()]
    dft_ct.reset_index(drop = True, inplace= True)
    # dfm_st = df_m[df_m.TELEFONO.isna()]
    dfm_ct = df#[Data_winback.TELEFONO.notna()]
    dfm_ct.reset_index(drop = True, inplace= True)
    x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
    y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
    x2 = dft_ct['TelefonoMovilCtaCliente'].tolist()
    y2 = dfm_ct['TELEFONO_MOVIL_H'].tolist()
    ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
    ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
    indx = list(set(ind_doc) & set(ind_con))
    # print(len(ind_doc))
    # nuevos
    df_New = dfm_ct.drop(ind_doc, axis=0)
    df['toque_'+y+'_'+m] = 0
    df['toque_'+y+'_'+m].iloc[indx] = 1
    # df_New = dfm_ct.drop(indx, axis=0)
    print('Registros nuevos: {}'.format(df_New.shape))
    i+=1

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Registros nuevos: (15999, 10)
Registros nuevos: (15960, 11)


In [ ]:
#@title Comparación de la base de datos si se encuentra duplicidad entre df y callout
# dft_st = df_t[df_t_.TELEFONO.isna()]
dft_ct = Data_callout#[df.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
# dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df#[Data_winback.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DE_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
# x2 = dft_ct['TELEFONO'].tolist()
# y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
# ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
# indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc))
# nuevos
df_New = dfm_ct.drop(ind_doc, axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

0
Registros nuevos: (15999, 10)


In [ ]:
#@title Comparación de la base de datos si se encuentra duplicidad entre df y ETB Total
# dft_st = df_t[df_t_.TELEFONO.isna()]
dft_ct = Data_ETB_Total#[Data_ETB_Total.TELEFONO.notna()][Data_ETB_Total.Emergia_Mayo_Bajas!=1][Data_ETB_Total.Emergia_Junio_Bajas!=1][Data_ETB_Total.Emergia_Julio_Bajas!=1][Data_ETB_Total.Emergia_Agosto_Bajas!=1][Data_ETB_Total.Emergia_Septiembre_Bajas!=1][Data_ETB_Total.Emergia_Octubre_Bajas!=1][Data_ETB_Total.Emergia_Noviembre_Bajas!=1][Data_ETB_Total.Emergia_Diciembre_Bajas!=1][Data_ETB_Total.Emergia_Enero23_Bajas!=1][Data_ETB_Total.Emergia_Febrero23_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
# dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df#[Data_winback.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO_MOVIL_H'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc))
# nuevos
# df_New = dfm_ct.drop(ind_doc, axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

## Verificación con Nro de documento y Telefono

In [ ]:
Data_winback.TELEFONO_PORTADO = Data_winback.TELEFONO_PORTADO.astype(str)
Data_winback.TELEFONO_PORTADO.apply(lambda x: len(x)).value_counts()

In [ ]:
#@title Comparación de la base de datos si se encuentra duplicidad entre df y winback
# dft_st = df_t[df_t_.TELEFONO.isna()]
dft_ct = Data_winback#[df.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
# dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df#[Data_winback.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO_PORTADO'].tolist()
y2 = dfm_ct['TELEFONO_MOVIL_H'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
indx = list(set(ind_doc) & set(ind_con))
print(len(ind_doc), len(ind_con), len(indx))
# nuevos
df_New = dfm_ct.drop(indx, axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

385 313 262
Registros nuevos: (15737, 10)


In [ ]:
Data_callout.CELULAR_ADIC_1

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
             ...     
74995             NaN
74996    3.112825e+09
74997             NaN
74998    3.016771e+09
74999    3.213796e+09
Name: CELULAR_ADIC_1, Length: 75000, dtype: float64

In [ ]:
Data_callout.CELULAR_ADIC_1 = Data_callout.CELULAR_ADIC_1.astype(str)
Data_callout.CELULAR_ADIC_1.apply(lambda x: len(x)).value_counts()

3     50383
12    24617
Name: CELULAR_ADIC_1, dtype: int64

In [ ]:
#@title Comparación de la base de datos si se encuentra duplicidad entre df y callout
# dft_st = df_t[df_t_.TELEFONO.isna()]
dft_ct = Data_callout#[df.TELEFONO.notna()]
dft_ct.reset_index(drop = True, inplace= True)
# dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df#[Data_winback.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DE_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
# x2 = dft_ct['TELEFONO'].tolist()
# y2 = dfm_ct['TELEFONO'].tolist()
ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)) , assume_unique=True))[0])
# ind_con = list(np.where(np.isin(y2, list(set(x2) & set(y2)) , assume_unique=True))[0])
# indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc),len(ind_con),len(indx))
# nuevos
df_New = dfm_ct.drop(ind_doc, axis=0)
print('Registros nuevos: {}'.format(df_New.shape))

Registros nuevos: (16000, 9)


In [ ]:
df.columns

Index(['NUMERO_DOCUMENTO', 'MarcaPlantaFija', 'FlagPlantaFija',
       'MarcaPlantaMovil', 'FlagPlantaMovil', 'MarcaHabeasData', 'HABEAS_DATA',
       'TELEFONO_MOVIL_H', 'NOMBRE_CLIENTE', 'Tam_num'],
      dtype='object')

In [ ]:
Data_ETB_Total.celular_1

0                   NaN
1          3.105707e+09
2          3.008288e+09
3          3.188269e+09
4                   NaN
               ...     
1151053             NaN
1151054             NaN
1151055             NaN
1151056             NaN
1151057    3.115314e+09
Name: celular_1, Length: 1151058, dtype: float64

In [ ]:
#@title Comparación de la base de datos si se encuentra duplicidad entre df y ETB Total
# dft_st = df_t[df_t_.TELEFONO.isna()]
dft_ct = Data_ETB_Total#[Data_ETB_Total.TELEFONO.notna()][Data_ETB_Total.Emergia_Mayo_Bajas!=1][Data_ETB_Total.Emergia_Junio_Bajas!=1][Data_ETB_Total.Emergia_Julio_Bajas!=1][Data_ETB_Total.Emergia_Agosto_Bajas!=1][Data_ETB_Total.Emergia_Septiembre_Bajas!=1][Data_ETB_Total.Emergia_Octubre_Bajas!=1][Data_ETB_Total.Emergia_Noviembre_Bajas!=1][Data_ETB_Total.Emergia_Diciembre_Bajas!=1][Data_ETB_Total.Emergia_Enero23_Bajas!=1][Data_ETB_Total.Emergia_Febrero23_Bajas!=1]
dft_ct.reset_index(drop = True, inplace= True)
# dfm_st = df_m[df_m.TELEFONO.isna()]
dfm_ct = df#[Data_winback.TELEFONO.notna()]
dfm_ct.reset_index(drop = True, inplace= True)
x1 = dft_ct['NUMERO_DOCUMENTO'].tolist()
y1 = dfm_ct['NUMERO_DOCUMENTO'].tolist()
x2 = dft_ct['TELEFONO'].tolist()
y2 = dfm_ct['TELEFONO_MOVIL_H'].tolist()
# ind_doc = list(np.where(np.isin(y1, list(set(x1) & set(y1)), assume_unique=True))[0])
ind_con = list(np.where(np.isin(x2, list(set(x2) & set(y2)), assume_unique=True))[0])
# indx = list(set(ind_doc) & set(ind_con))
# print(len(ind_doc))
print(len(ind_con))
# nuevos
# df_New = dfm_ct.drop(indx, axis=0)
# print('Registros nuevos: {}'.format(df_New.shape))